In [43]:
import scanpy as sc
import numpy as np
import os
import anndata2ri
import pathlib
import scvelo as scv
from scipy import io
import anndata#
import pandas as pd
from tqdm import tqdm
import argparse
import sys

# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
# sc.logging.print_versions()

import warnings
warnings.filterwarnings("ignore")

all_datasets = ["220907_FH",
                "GSM5764259",
                "230228_FH",
                "GSM5764288",            
                "GSM5764245",
                "integrate_GSE192742_LIVER",
                "gutcellatlas_myeloid"]

outdir = "/media/hieunguyen/HNSD01/outdir"

for orig_dataset in tqdm(all_datasets):
# for orig_dataset in ["220907_FH"]:
    config_version = "v0.1"
    output_version = "20240806"
    PROJECT = "FHager_datasets"
    
    loom_dir = "/media/hieunguyen/HNSD01/storage/FHager_datasets/velocyto_output"
    
    dataset_name = "{}_{}".format(orig_dataset, config_version)
    colorfile = os.path.join("/media/hieunguyen/HNSD01/outdir/FHager_datasets/20240806/colors", dataset_name, "colordf.csv")
    colors = pd.read_csv(colorfile)["color"].to_list()
    path_to_main_input = os.path.join(outdir,
                                    PROJECT,
                                    output_version, 
                                    dataset_name, 
                                    "s8a_output",
                                    "{}.output.s8a.rds".format(dataset_name))
    
    path_to_seurat2anndata = os.path.join(outdir, PROJECT, output_version, "seurat2anndata", dataset_name)
    path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
    path_to_05_output = os.path.join(path_to_main_output, "05_output")
    os.system("mkdir -p {}".format(path_to_05_output))
    
    
    adata = sc.read_h5ad(os.path.join(path_to_seurat2anndata, '{}.h5ad'.format(dataset_name)))
    loom_data = scv.read_loom(os.path.join(loom_dir, "{}.loom".format(orig_dataset)))
    if "obs_names" in loom_data.obs.columns:
        loom_data.obs.index = loom_data.obs.obs_names.to_list()
    
    # print(orig_dataset)
    # print(adata.obs.head().index)
    # print(loom_data.obs.head().index)

    if orig_dataset == "220907_FH":
        adata_new_obs = [item.split("_")[-1].split("-")[0] for item in adata.obs.index]
        loom_new_obs = [item.replace("LKopplin_220907_with_EGFP_SnapGene:", "").replace("x", "") for item in loom_data.obs.index]
        # assert len([item for item in adata_new_obs if item in loom_new_obs]) == len(adata_new_obs)
        
    elif orig_dataset == "GSM5764259":
        adata_new_obs = [item.replace(".1", "")for item in adata.obs.index]
        loom_new_obs = [item.replace("Sample_GSM5764259:", "").replace("x", "") for item in loom_data.obs.index]
        # assert len([item for item in adata_new_obs if item in loom_new_obs]) == len(c)
    elif orig_dataset == "230228_FH":
        adata_new_obs = [item.replace("230228_Kopplin_Pabst_", "")for item in adata.obs.index]
        loom_new_obs = [item.replace("LKopplin_230228_with_EGFP_SnapGene:", "").replace("x", "") for item in loom_data.obs.index]
        # assert len([item for item in adata_new_obs if item in loom_new_obs]) == len(adata_new_obs)
    elif orig_dataset == "GSM5764288":
        adata_new_obs = [item.replace(".1", "")for item in adata.obs.index]
        loom_new_obs = [item.replace("Sample_GSM5764288:", "").replace("x", "") for item in loom_data.obs.index]
        # assert len([item for item in adata_new_obs if item in loom_new_obs]) == len(adata_new_obs)
    elif orig_dataset == "GSM5764245":
        adata_new_obs = [item.replace(".1", "")for item in adata.obs.index]
        loom_new_obs = [item.replace("Sample_GSM5764245:", "").replace("x", "") for item in loom_data.obs.index]
        # assert len([item for item in adata_new_obs if item in loom_new_obs]) == len(adata_new_obs)
    elif orig_dataset == "integrate_GSE192742_LIVER":
        adata_new_obs = [item.split("_")[-1].split("-")[0] for item in adata.obs.index]
        loom_new_obs = [item.split("_")[-1].split("-")[0] for item in loom_data.obs.index]
        # assert len([item for item in adata_new_obs if item in loom_new_obs]) == len(adata_new_obs)
    elif orig_dataset == "gutcellatlas_myeloid":
        adata_new_obs = adata.obs.index
        loom_new_obs = loom_data.obs.index
        # assert len([item for item in adata.obs.index if item in loom_data.obs.index]) == len(adata.obs.index)
    
    adata.obs.index = adata_new_obs
    loom_data.obs.index = loom_new_obs
    subset_adata = adata[[item for item in adata_new_obs if item in loom_new_obs]]
    subset_loom_data = loom_data[[item for item in adata_new_obs if item in loom_new_obs]]
    merge_data = scv.utils.merge(subset_adata, subset_loom_data)

    scv.pp.filter_genes(merge_data, min_shared_counts=20)
    scv.pp.normalize_per_cell(merge_data)
    scv.pp.filter_genes_dispersion(merge_data, n_top_genes=2000)
    scv.pp.log1p(merge_data)
    
    scv.pp.filter_and_normalize(merge_data, min_shared_counts=20, n_top_genes=2000)
    scv.pp.moments(merge_data, n_pcs=30, n_neighbors=30)

    ##### Running RNA velocity inference
    scv.tl.recover_dynamics(merge_data, n_jobs = 15)
    scv.tl.velocity(merge_data, diff_kinetics=True, mode="dynamical")
    scv.tl.velocity_graph(merge_data)

    ##### arrows velocity
    scv.pl.velocity_embedding(merge_data, dpi=120, arrow_size=2, arrow_length=10, basis = "X_umap",color="seurat_clusters",
                         figsize = (12, 12), fontsize=20, legend_fontsize=40, frameon=True, palette = colors, save="arrows_{}.svg".format(dataset_name))

    ##### streamline velocity plot
    scv.pl.velocity_embedding_stream(merge_data, dpi=120, arrow_size=2, basis = "X_umap",color="seurat_clusters",
                         figsize = (12, 12), fontsize=20, legend_fontsize = 30, frameon=True, palette = colors, save="streamline_{}.svg".format(dataset_name))

    ##### pseuotime plot
    scv.tl.velocity_pseudotime(merge_data)
    scv.pl.scatter(merge_data, color='velocity_pseudotime', cmap='gnuplot', basis = "X_umap",
                   figsize = (15, 15), fontsize=20, legend_fontsize = 30, frameon=True, save="pseudotime_{}.svg".format(dataset_name))

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


  0%|                                                     | 0/7 [00:02<?, ?it/s]

KeyboardInterrupt

